# Import

In [1]:
import pandas as pd
import numpy as np
import glob
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from tqdm import tqdm

# Load Validation / Test dataset

In [3]:
# val = pd.read_csv("./sample_train.csv") # 경로로부터 sample_train.csv파일을 읽어옵니다.
# val.head()
val = pd.read_csv("C:\\Users\\82102\\Desktop\\Machine_Learning\\DACON_materials\\Code_Similarity\\sample_train.csv")
val.head()

,code1_path,code2_path,code1,code2,similar
0,./train_code/problem393/problem393_19.cpp,./train_code/problem033/problem033_439.cpp,#include <bits/stdc++.h>\n\nusing namespace st...,#include <algorithm>\n#include <bitset>\n#incl...,0
1,./train_code/problem019/problem019_210.cpp,./train_code/problem019/problem019_63.cpp,#include <iostream>\n\nusing namespace std;\n\...,#include <iostream>\n#include <string>\nusing ...,1
2,./train_code/problem107/problem107_486.cpp,./train_code/problem107/problem107_340.cpp,#include <iostream>\n#include <vector>\nusing ...,#include <cstdio>\n#include <cstdlib>\n#includ...,1
3,./train_code/problem187/problem187_257.cpp,./train_code/problem403/problem403_135.cpp,#include <bits/stdc++.h>\n#include <unordered_...,#include <bits/stdc++.h>\nusing namespace std;...,0
4,./train_code/problem173/problem173_490.cpp,./train_code/problem173/problem173_345.cpp,#include <bits/stdc++.h>\ntypedef long long ll...,"#include ""bits/stdc++.h""\n#define rep(i,n) for...",1


In [4]:
# test = pd.read_csv("./test.csv") # 경로로부터 test.csv파일을 읽어옵니다.
# test.head()
test = pd.read_csv("C:\\Users\\82102\\Desktop\\Machine_Learning\\DACON_materials\\Code_Similarity\\test.csv")
test.head()

,pair_id,code1,code2
0,TEST_000000,#include <bits/stdc++.h>\nusing namespace std;...,"#include <bits/stdc++.h>\n#define rep(i, n) fo..."
1,TEST_000001,"#include<bits/stdc++.h>\n#define rep(i,n)for(i...",// //bitset操作\n// #include <iostream>\n// #inc...
2,TEST_000002,#include <bits/stdc++.h>\nusing namespace std;...,#include <bits/stdc++.h>\n#include <ext/pb_ds/...
3,TEST_000003,#include <bits/stdc++.h>\nusing namespace std;...,#include <bits/stdc++.h>\nusing namespace std;...
4,TEST_000004,#include<bits/stdc++.h>\nusing namespace std;\...,#include<iostream>\n#include<algorithm>\n#incl...


# Define Model (CountVectorizer+CosineSimilarity)

1. CountVectorizer: 코드 데이터를 활용하기 위해 코드(Text)를 벡터화하여 사용합니다. 이를 위해 베이스라인에서는 sklearn 라이브러리의 CountVectorizer를 사용합니다.
2. fit함수: 입력받은 학습 코드로 부터 vectorizer를 모델에 fit시킵니다. 본 베이스라인에서는 fit 호출마다 vectorizer에 활용할 vocabulary를 업데이트합니다.
3. predict_proba 함수: 입력받은 코드 쌍으로부터 vectorizer를 통해 vector화를 수행하고, 코사인 유사도를 구합니다.
4. predict 함수: 유사도를 계산하고, threshold를 기준으로 판별하여 예측을 수행합니다.을 수행합니다.

In [5]:
class BaselineModel():
    def __init__(self, threshold=0.5):
        super(BaselineModel, self).__init__()
        self.threshold = threshold # 유사도 임계값 설정
        self.vocabulary = set() # 사용할 어휘를 저장할 집합 초기화

    def get_vectorizer(self):
        return CountVectorizer(vocabulary=list(self.vocabulary)) # 현재 저장된 단어 목록을 기반으로 vectorizer 생성합니다.
        
    def fit(self, code):
        # 입력받은 학습 코드로부터 vectorizer를 fit 시킵니다.
        temp_vectorizer = CountVectorizer() # 임시 vectorizer 생성합니다.
        temp_vectorizer.fit(code) # code 데이터로 vectorizer 학습합니다.
        # fit 호출 마다 vectorizer에 활용할 vocabulary 업데이트합니다.
        self.vocabulary.update(temp_vectorizer.get_feature_names_out()) # 학습된 어휘를 기존 어휘에 추가합니다.
        # fit 호출 마다 vectorizer 업데이트합니다.
        self.vectorizer = self.get_vectorizer()
    
    def predict_proba(self, code1, code2):
        # 입력받은 코드 쌍으로부터 vectorizer를 통해 vector화 합니다.
        code1_vecs = self.vectorizer.transform(code1) # 첫 번째 코드를 벡터로 변환합니다.
        code2_vecs = self.vectorizer.transform(code2) # 두 번째 코드를 벡터로 변환합니다.
        
        preds = []
        # 각각의 코드 쌍(=벡터 쌍)으로부터 cosine-similarity를 구합니다.
        for code1_vec, code2_vec in tqdm(zip(code1_vecs, code2_vecs)):
            preds.append(cosine_similarity(code1_vec, code2_vec)) # 각 코드 쌍에 대해 코사인 유사도 계산 후 추가합니다.
        preds = np.reshape(preds, len(preds))
        print('Done.')
        return preds
    
    def predict(self, code1, code2):
        preds = self.predict_proba(code1, code2) # 유사도 계산합니다.
        preds = np.where(preds>self.threshold, 1, 0) # threshold를 기준으로 판별합니다.
        return preds

# Model(Vectorizer) Fit

In [6]:
train_code_paths = glob.glob('C:/Users/82102/Desktop/Machine_Learning/DACON_materials/Code_Similarity/train_code/*/*.cpp') # train code의 모든 .cpp 파일의 경로를 가져옵니다.

In [7]:
def read_cpp_code(file_path): 
    with open(file_path, 'r', encoding='utf-8') as file:
        return file.read() # 주어진 경로 내 파일을 열고 파일의 전체 내용을 반환합니다.

In [8]:
# threshold(임계값)을 설정하고, 모델을 선언합니다.
model = BaselineModel(threshold=0.5)

In [9]:
# 진행 상황을 표시하는 바와 함께 모든 파일 경로에 대해 반복 학습합니다.
for path in tqdm(train_code_paths):
    code = read_cpp_code(path)
    model.fit([code]) # 읽어온 코드를 리스트로 묶어 모델에 학습시킵니다.

100%|█████████████████████████████████████████████████████████████████████████| 250000/250000 [33:51<00:00, 123.05it/s]


In [10]:
len(model.vocabulary) # 모델에 구축된 vocabulary 크기를 출력합니다.

113727

# Validation

In [11]:
def get_accuracy(gt, preds): # 모델의 정확도 측정을 위한 함수입니다.
    return (gt == preds).mean()

In [12]:
val_preds = model.predict(val['code1'], val['code2']) # 검증 데이터셋을 활요한 예측 수행합니다.

20000it [00:18, 1060.05it/s]

Done.


In [13]:
print(get_accuracy(val['similar'].values, val_preds)) # 정확도를 출력합니다.

0.60125


# Inference

In [14]:
# 모델 추론
preds = model.predict(test['code1'], test['code2'])

595000it [09:13, 1074.74it/s]


Done.


# Submissionn

In [16]:
# 제출

submission = pd.read_csv("C:/Users/82102/Desktop/Machine_Learning/DACON_materials/Code_Similarity/sample_submission.csv")
submission['similar'] = preds
submission.to_csv('C:/Users/82102/Desktop/Machine_Learning/DACON_materials/Code_Similarity/base_submit.csv', index=False)

### **점수: 0.60772**